In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Analytics Vidya Customer segmentation/Train_aBjfeNk.csv')
test = pd.read_csv('/content/drive/My Drive/Analytics Vidya Customer segmentation/Test_LqhgPWU.csv')
subs = pd.read_csv('/content/drive/My Drive/Analytics Vidya Customer segmentation/sample_submission_wyi0h0z.csv')

In [ ]:
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [ ]:
from sklearn.preprocessing import LabelEncoder
lb1 = LabelEncoder()
lb = LabelEncoder()

In [ ]:
train_copy  = train.copy()
test_copy = test.copy()
train_copy['flag'] = 1
test_copy['flag'] = 0

total = pd.concat([train_copy, test_copy], axis = 0)
from sklearn.cluster import KMeans
km = KMeans(n_clusters=4)
k_train_test = km.fit_predict(total[['Age']])
total['Age_score'] = k_train_test

In [ ]:
cat_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
label_enc = {}
for col in cat_cols:
    total[col] = total[col].astype(str)
    enc = LabelEncoder().fit(total[col])
    total[col] = enc.transform(total[col])
    label_enc[col] = enc

In [ ]:
train.shape , test.shape

((8068, 11), (2627, 10))

In [ ]:
total = pd.get_dummies(total, columns = cat_cols)

In [ ]:

train_copy = total.loc[total['flag'] == 1]
test_copy = total.loc[total['flag'] == 0]
Xcols = total.drop(['Segmentation', 'flag','Age'], axis = 1).columns
ycol = 'Segmentation'

X = train_copy[Xcols]
y = train_copy[ycol]

Xtest = test_copy[Xcols]

In [ ]:
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import xgboost as xgb
from sklearn.pipeline import make_pipeline
model = make_pipeline(SimpleImputer(strategy='most_frequent'), StandardScaler(),
                      xgb.XGBClassifier(objective="multi:softmax",n_jobs=-1,num_class=4,eval_metric="auc",
                                                      learning_rate =0.1,
                                                         n_estimators=1000,
                                                         max_depth=9,
                                                         min_child_weight=1,
                                                         gamma=0,
                                                         subsample=0.8,
                                                         colsample_bytree=0.8,random_state=43)).fit(X, y)

In [ ]:
y_pred_pipe = model.predict(Xtest)

In [ ]:
subs['ID'] = test.ID
subs["Segmentation"]=y_pred_pipe
subs.to_csv("/content/drive/My Drive/Analytics Vidya Customer segmentation/acc.csv",index=False)